In [ ]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dropout, LSTM, Flatten, Dense

In [ ]:
df_train = pd.read_csv('ECG_Data_Lead_csv/11_labels/correct_train.csv')
df_val = pd.read_csv('ECG_Data_Lead_csv/11_labels/correct_val.csv')
df_test = pd.read_csv('ECG_Data_Lead_csv/11_labels/correct_test.csv')

In [ ]:
print(f"Train shape: {df_train.shape}, Val shape: {df_val.shape}, Test shape: {df_test.shape}")

In [ ]:
X_train = df_train.iloc[:, :700].values
X_train_norm = X_train / np.max(np.abs(X_train))
X_train_norm = X_train_norm.reshape(X_train_norm.shape[0], 700, 1)
y_train = df_train['label'].values

X_val = df_val.iloc[:, :700].values
X_val_norm = X_val / np.max(np.abs(X_train))
X_val_norm = X_val_norm.reshape(X_val_norm.shape[0], 700, 1)
y_val = df_val['label'].values


X_test = df_test.iloc[:, :700].values
X_test_norm = X_test / np.max(np.abs(X_train))
X_test_norm = X_test_norm.reshape(X_test_norm.shape[0], 700, 1)
y_test = df_test['label'].values

In [ ]:
scaler = StandardScaler()

X_train_norm = scaler.fit_transform(X_train)

X_val_norm = scaler.transform(X_val)
X_test_norm = scaler.transform(X_test)

In [ ]:
print(f"X_train_norm max: {X_train_norm.max()}, min: {X_train_norm.min()}, mean: {np.mean(X_train_norm)}, std: {np.std(X_train_norm)}")
print(f"X_val_norm max: {X_val_norm.max()}, min: {X_val_norm.min()}, mean: {np.mean(X_val_norm)}, std: {np.std(X_val_norm)}")
print(f"X_test_norm max: {X_test_norm.max()}, min: {X_test_norm.min()}, mean: {np.mean(X_test_norm)}, std: {np.std(X_test_norm)}")


In [ ]:
model = tf.keras.models.Sequential([
    Conv1D(filters=64, kernel_size=(5), activation='relu', input_shape=(700,1)),
    MaxPooling1D(pool_size=2),
    Dropout(0.2),

    Conv1D(filters=64, kernel_size=(3), activation='relu'), 
    LSTM(units=64, return_sequences=True),
    Flatten(),
    
    Dense(64, activation='relu'),
    Dense(11, activation='softmax')
])


In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train_norm, y_train, epochs=10, validation_data=(X_val_norm, y_val))